In [1]:
from config import *
from utilities import get_model_name
from dataset import ClassificationDataset

import functools
import numpy as np
import pandas as pd

results_dict = {}
internal, external = 11, 12
encode_method = "dummy"
impute_method = "knn7u" #TODO
fs_method, fs_ratio = "dnp", 0.5  #dnp & graces
norm_method = "maxmin"
classification_dataset = ClassificationDataset(
    internal, external, encode_method, impute_method, fs_method, fs_ratio, norm_method, random_state=SEED)

Number of Selected Features: 100%|██████████| 71/71 [06:26<00:00,  5.45s/it]


In [2]:
# from sklearn.neural_network import MLPClassifier
from utilities import MLPClassifier

model_fn = functools.partial(
    MLPClassifier, random_state=SEED, max_iter=50,
    momentum=0.9, squares=0.999, optimizer_t='sgdm',
    use_residual=True, use_batch_norm=False)
param_grid_list = [{
    # hidden_layer_sizes = (64, 32)
    # config       lr_range            worst_auc
    # adam w/ bn:  [2e-5, 5e-5, 1e-4]  0.8942
    # adam w/o bn: [5e-5, 1e-4, 2e-4]  0.9148
    # sgdm w/ bn:  [1e-3, 2e-3, 5e-3]  0.8982
    # sgdm w/o bn: [5e-3, 1e-2, 2e-2]  0.9177

    'hidden_layer_sizes': [(128, 128), (64, 64, 64)],
    'learning_rate': [5e-3, 1e-2, 2e-2],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

MLPClassifier


Epoch 49/50:   0%|          | 0/26 [00:00<?, ?batch/s, loss=0.131]           

model(hidden_layer_sizes=(128, 128), learning_rate=0.005)
Valid:         AUC: 0.9188 | AUPR: 0.6586 | Acc: 95.41%


model(hidden_layer_sizes=(128, 128), learning_rate=0.01)
Valid:         AUC: 0.9216 | AUPR: 0.6651 | Acc: 95.39%


model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9231 | AUPR: 0.6670 | Acc: 95.24%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.005)
Valid:         AUC: 0.9172 | AUPR: 0.6568 | Acc: 95.38%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.01)
Valid:         AUC: 0.9206 | AUPR: 0.6618 | Acc: 95.41%


model(hidden_layer_sizes=(64, 64, 64), learning_rate=0.02)
Valid:         AUC: 0.9222 | AUPR: 0.6666 | Acc: 95.29%
best model(hidden_layer_sizes=(128, 128), learning_rate=0.02)
Valid:         AUC: 0.9231 | AUPR: 0.6670 | Acc: 95.24%
Internal test: AUC: 0.9287 | AUPR: 0.6965 | Acc: 95.06%
External test: AUC: 0.9217 | AUPR: 0.6662 | Acc: 95.15%


In [3]:
from sklearn.svm import SVC

model_fn = functools.partial(
    SVC, random_state=SEED, probability=True,
    class_weight='balanced')
param_grid_list = [{
    'kernel': ['linear'],
    'C': [0.01, 0.1, 1],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

SVC
model(kernel=linear, C=0.01)
Valid:         AUC: 0.9178 | AUPR: 0.6269 | Acc: 95.05%
model(kernel=linear, C=0.1)
Valid:         AUC: 0.9174 | AUPR: 0.6216 | Acc: 94.93%
model(kernel=linear, C=1)
Valid:         AUC: 0.9163 | AUPR: 0.6176 | Acc: 94.94%
best model(kernel=linear, C=0.01)
Valid:         AUC: 0.9178 | AUPR: 0.6269 | Acc: 95.05%
Internal test: AUC: 0.9229 | AUPR: 0.6631 | Acc: 94.65%
External test: AUC: 0.9153 | AUPR: 0.6094 | Acc: 94.62%


In [4]:
from sklearn.neighbors import KNeighborsClassifier

model_fn = functools.partial(
    KNeighborsClassifier, n_jobs=-1, algorithm='auto', p=2)
param_grid_list = [{
    'n_neighbors': [3, 5, 7],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

KNeighborsClassifier
model(n_neighbors=3)
Valid:         AUC: 0.7596 | AUPR: 0.4283 | Acc: 94.69%
model(n_neighbors=5)
Valid:         AUC: 0.7877 | AUPR: 0.4770 | Acc: 95.02%
model(n_neighbors=7)
Valid:         AUC: 0.8015 | AUPR: 0.4968 | Acc: 95.08%
best model(n_neighbors=7)
Valid:         AUC: 0.8015 | AUPR: 0.4968 | Acc: 95.08%
Internal test: AUC: 0.8014 | AUPR: 0.5393 | Acc: 94.91%
External test: AUC: 0.8168 | AUPR: 0.5163 | Acc: 94.84%


In [5]:
from sklearn.naive_bayes import GaussianNB

model_fn = GaussianNB
param_grid_list = [{}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GaussianNB
model()
Valid:         AUC: 0.8795 | AUPR: 0.4488 | Acc: 83.47%
best model()
Valid:         AUC: 0.8795 | AUPR: 0.4488 | Acc: 83.47%
Internal test: AUC: 0.8867 | AUPR: 0.4887 | Acc: 84.23%
External test: AUC: 0.8762 | AUPR: 0.4291 | Acc: 82.59%


In [6]:
# from sklearn.exceptions import ConvergenceWarning
# import warnings
# warnings.filterwarnings("ignore", category=ConvergenceWarning)
from sklearn.linear_model import LogisticRegression

model_fn = functools.partial(
    LogisticRegression, random_state=SEED, n_jobs=-1, 
    max_iter=2000, solver='saga', penalty='l2',
    )
param_grid_list = [{
    'C': [1.0, 10.0, 100.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(
    model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(
    best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

LogisticRegression


model(C=1.0)
Valid:         AUC: 0.9188 | AUPR: 0.6537 | Acc: 95.33%
model(C=10.0)
Valid:         AUC: 0.9184 | AUPR: 0.6526 | Acc: 95.33%
model(C=100.0)
Valid:         AUC: 0.9178 | AUPR: 0.6521 | Acc: 95.33%
best model(C=1.0)
Valid:         AUC: 0.9188 | AUPR: 0.6537 | Acc: 95.33%
Internal test: AUC: 0.9252 | AUPR: 0.6868 | Acc: 95.13%
External test: AUC: 0.9133 | AUPR: 0.6333 | Acc: 94.90%


In [7]:
from sklearn.tree import DecisionTreeClassifier

model_fn = functools.partial(
    DecisionTreeClassifier, random_state=SEED,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

DecisionTreeClassifier
model(criterion=gini)
Valid:         AUC: 0.8687 | AUPR: 0.5563 | Acc: 83.66%
model(criterion=log_loss)
Valid:         AUC: 0.8726 | AUPR: 0.5725 | Acc: 82.77%
model(criterion=entropy)
Valid:         AUC: 0.8726 | AUPR: 0.5725 | Acc: 82.77%
best model(criterion=log_loss)
Valid:         AUC: 0.8726 | AUPR: 0.5725 | Acc: 82.77%
Internal test: AUC: 0.8811 | AUPR: 0.6235 | Acc: 85.35%
External test: AUC: 0.8807 | AUPR: 0.5947 | Acc: 85.12%


In [8]:
from sklearn.ensemble import RandomForestClassifier

model_fn = functools.partial(
    RandomForestClassifier, random_state=SEED, n_jobs=-1,
    ccp_alpha=0.0, class_weight='balanced', max_depth=5, bootstrap=False)
param_grid_list = [{
        'criterion': ['gini', 'log_loss', 'entropy'],
        'n_estimators': [50, 100, 200],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model_fn, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

RandomForestClassifier
model(criterion=gini, n_estimators=50)
Valid:         AUC: 0.9127 | AUPR: 0.6519 | Acc: 88.95%
model(criterion=gini, n_estimators=100)
Valid:         AUC: 0.9138 | AUPR: 0.6527 | Acc: 89.06%
model(criterion=gini, n_estimators=200)
Valid:         AUC: 0.9141 | AUPR: 0.6531 | Acc: 89.01%
model(criterion=log_loss, n_estimators=50)
Valid:         AUC: 0.9129 | AUPR: 0.6554 | Acc: 88.90%
model(criterion=log_loss, n_estimators=100)
Valid:         AUC: 0.9140 | AUPR: 0.6551 | Acc: 89.17%
model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9148 | AUPR: 0.6562 | Acc: 89.27%
model(criterion=entropy, n_estimators=50)
Valid:         AUC: 0.9129 | AUPR: 0.6554 | Acc: 88.90%
model(criterion=entropy, n_estimators=100)
Valid:         AUC: 0.9140 | AUPR: 0.6551 | Acc: 89.17%
model(criterion=entropy, n_estimators=200)
Valid:         AUC: 0.9148 | AUPR: 0.6562 | Acc: 89.27%
best model(criterion=log_loss, n_estimators=200)
Valid:         AUC: 0.9148 | AUPR: 0.6562 | Ac

In [9]:
from sklearn.ensemble import GradientBoostingClassifier

model = functools.partial(
    GradientBoostingClassifier, random_state=SEED,
    learning_rate=0.1, max_depth=5, loss='log_loss', 
    n_estimators=100)
param_grid_list = [{
        'criterion': ['friedman_mse', 'squared_error'],
    }
]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

GradientBoostingClassifier
model(criterion=friedman_mse)
Valid:         AUC: 0.9192 | AUPR: 0.6522 | Acc: 95.22%
model(criterion=squared_error)
Valid:         AUC: 0.9188 | AUPR: 0.6514 | Acc: 95.19%
best model(criterion=friedman_mse)
Valid:         AUC: 0.9192 | AUPR: 0.6522 | Acc: 95.22%
Internal test: AUC: 0.9289 | AUPR: 0.7017 | Acc: 95.30%
External test: AUC: 0.9221 | AUPR: 0.6668 | Acc: 95.14%


In [10]:
from sklearn.ensemble import AdaBoostClassifier
model = functools.partial(AdaBoostClassifier, random_state=SEED, algorithm="SAMME")
param_grid_list = [{
    'n_estimators': [100, 200, 500],
    'learning_rate': [1.0],
}]
best_metrics, best_params, best_model = classification_dataset.perform_grid_search(model, param_grid_list)
metrics_internal, metrics_external = classification_dataset.evaluate_test_sets(best_model)
results_dict[get_model_name(model_fn)] = np.concatenate([best_metrics, metrics_internal, metrics_external], axis=0)

AdaBoostClassifier


model(n_estimators=100, learning_rate=1.0)
Valid:         AUC: 0.9144 | AUPR: 0.6260 | Acc: 95.11%
model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9157 | AUPR: 0.6244 | Acc: 95.04%
model(n_estimators=500, learning_rate=1.0)
Valid:         AUC: 0.9151 | AUPR: 0.6230 | Acc: 95.11%
best model(n_estimators=200, learning_rate=1.0)
Valid:         AUC: 0.9157 | AUPR: 0.6244 | Acc: 95.04%
Internal test: AUC: 0.9191 | AUPR: 0.6812 | Acc: 95.21%
External test: AUC: 0.9154 | AUPR: 0.6207 | Acc: 94.75%


In [11]:
dataframe = pd.DataFrame(results_dict).transpose().reset_index()
dataframe.columns = ['model', 'val_auc', 'val_aupr', 'val_acc', 'intest_auc', 'intest_aupr', 'intest_acc', 'extest_auc', 'extest_aupr', 'extest_acc']
# dataframe.to_csv()
filename = os.path.join(RESULTS_DIR, encode_method, impute_method, f'{fs_method}{int(fs_ratio * 142)}.csv')
dirname = os.path.dirname(filename)
if not os.path.exists(dirname):
    os.makedirs(dirname)
dataframe.to_csv(filename, index=False)